# **SAVING AND LOADING A MODEL**

Training a neural network is hard work and time-consuming. Tat's why retraining a
model every time is impractical. Te good news is that we can save a network to disk and
load it whenever we need it, whether to improve its performance with more training or to
use it to make predictions on fresh data. In this recipe, we'll learn about diﬀerent ways to
persist a model.

# IMPORTING LIBRARIES

In [33]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Softmax
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json




#Defne a function that will download and prepare the data by normalizing the train and test sets and one-hot encoding the labels

In [34]:
def load_data():
  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  # NORMALIZE DATA
  x_train  = x_train.astype("float32") / 255.0
  x_test = x_test.astype("float32") / 255.0

  # reshape grayscale to include channel dimension
  x_train = np.expand_dims(x_train, axis=3)
  x_test = np.expand_dims(x_test, axis=3)

  # Process labels
  label_binarizer = LabelBinarizer()
  y_train = label_binarizer.fit_transform(y_train)
  y_test = label_binarizer.transform(y_test)

  return x_train, y_train, x_test, y_test


# Defne a function for building a network. Te architecture comprises a single convolutional layer and two fully connected layers:

In [41]:
def build_network():
  input_layer = Input(shape = (28, 28, 1))
  convolution_1 = Conv2D(kernel_size = (2, 2),
                         padding = "same",
                         strides = (2, 2),
                         filters = 32) (input_layer)
  activation_1 = ReLU() (convolution_1)
  batch_normalization_1 = BatchNormalization() (activation_1)
  pooling_1 = MaxPooling2D(pool_size = (2, 2), strides = (1, 1), padding = "same") (batch_normalization_1)
  dropout = Dropout(rate = 0.5) (pooling_1)

  flatten  = Flatten() (dropout)
  dense_1 = Dense(units = 128) (flatten)
  activation_2 = ReLU() (dense_1)
  dense_2 = Dense(units = 10) (activation_2)
  output = Softmax() (dense_2)

  network = Model(inputs = input_layer, outputs = output)
  return network

# Implement a function that will evaluate a network using the test set:

In [39]:
def evaluate(model, x_test, y_test):
  _, accuracy = model.evaluate(x_test, y_test, verbose = 0)
  print(f"accuracy: {accuracy}")

# Prepare the data, create a validation split, and instantiate the neural network:

In [42]:
X_train, y_train, X_test, y_test = load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8)
model = build_network()

# Compile and train the model for 50 epochs, with a batch size of 1024. Feel free to tune these values according to the capacity of your machine:

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=1024, verbose=0)


# Save the model, along with its weights, in HDF5 format using the save() method. Ten, load the persisted model using load_model() and evaluate the network's performance on the test set:

In [45]:
# Saving model and weights as HDF5.
model.save('model_and_weights.hdf5')

# Loading model and weights as HDF5.
loaded_model = load_model('model_and_weights.hdf5')

# Predicting using loaded model.
evaluate(loaded_model, X_test, y_test)

accuracy: 0.9840999841690063
